In [ ]:
# Install PyTorch dependencies
!pip install -q torch torchvision tqdm pillow

# Verify GPU
import torch
print(f"GPU available: {torch.cuda.is_available()}")
print(f"GPU name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

In [ ]:
# Set environment variables
import os
os.environ['SM_MODEL_DIR'] = './models4'
os.environ['SM_CHANNEL_TRAIN'] = '/kaggle/input/cleaned-cat-breed/cat-dataset/train'
os.environ['SM_CHANNEL_VALIDATION'] = '/kaggle/input/cleaned-cat-breed/cat-dataset/test'
os.environ['SM_OUTPUT_DATA_DIR'] = './output4'

# Create directories
!mkdir -p models4 output4

print("🚀 Starting GPU training...")
print("⏱️  Estimated time: ~6.5 hours\n")

# Run training as subprocess (avoids argparse conflict)
!python /kaggle/input/training/train.py \
  --model-dir ./models4 \
  --train /kaggle/input/cleaned-cat-breed/cat-dataset/train \
  --validation /kaggle/input/cleaned-cat-breed/cat-dataset/test \
  --output-data-dir ./output4 \
  --model efficientnet_b1 \
  --epochs 50 \
  --patience 12 \
  --progressive-resize \
  --mixup-alpha 0.7 \
  --cutmix-alpha 1.3 \
  --mixup-prob 0.8

print("\n✅ Training complete!")

In [ ]:

# Upload trained model back to S3
print("📤 Uploading model to S3...")

!aws s3 cp models/model.pth s3://feline-purrfect-ml/models/breed-detector/model.pth
!aws s3 cp models/classes.json s3://feline-purrfect-ml/models/breed-detector/classes.json
!!aws s3 cp output/training_history.json s3://feline-purrfect-ml/outputs/breed-detector/training_history.json

print("✅ Model uploaded to S3!")
print("Location: s3://feline-purrfect-ml/models/breed-detector/")